In [1]:
import pandas as pd
import numpy as np
import time
import os 
from pandas import Series, DataFrame
from tqdm.notebook import tqdm

dir_ = '../data/'
file_name = 'filter_track_5_user_100.pkl'

In [2]:
lh = pd.read_pickle(os.path.join(dir_, file_name))

In [4]:
unique_users = lh['uid'].unique()
unique_user = len(unique_users)
unique_tracks = lh['tid'].unique()
unique_track = len(unique_tracks)

print(unique_user)
print(unique_track)
print(len(lh))

220
54969
702362


In [22]:
lh[:10]

,uid,tid,count
4,0,4,15
5,0,5,3
6,0,6,1
7,0,7,1
8,0,8,10
9,0,9,1
10,0,10,2
11,0,11,3
13,0,13,2
14,0,14,1


In [23]:
track_keys = {}

for track in unique_tracks:
    track_keys[track] = len(track_keys)
    
user_keys = {}

for user in unique_users:
    user_keys[user] = len(user_keys)

In [24]:
lh['tid']= lh['tid'].map(track_keys)
lh['uid']= lh['uid'].map(user_keys)

In [25]:
lh[:10]

,uid,tid,count
4,0,0,15
5,0,1,3
6,0,2,1
7,0,3,1
8,0,4,10
9,0,5,1
10,0,6,2
11,0,7,3
13,0,8,2
14,0,9,1


In [26]:
threshold = []
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    user = user.sort_values(by=['count'])
    t = []
    for j in range(19):
        v = user.iloc[len(user)//20*(j+1)]['count']
        t.append(v)
    threshold.append(t)
threshold

[[1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 5, 5, 6, 8, 11, 16],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 7],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 8],
 [1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 8, 10, 12, 14, 18, 26],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 6],
 [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 6, 6, 7, 8, 10, 13],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 6, 8, 14, 22],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 4, 5, 6],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 8],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, 6],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 6, 8],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 4],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 7],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 7],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3],
 [1, 1, 2, 2, 2, 3, 3, 4, 5, 

In [27]:
a = threshold[0][18]
print(a)

16


In [28]:
start = time.time()
def to_rating(df):  
    rating = 0
    for i in range(18,-1,-1):
        if df['count'] <= threshold[df['uid']][i]:
            continue
        else:
            rating = i+1
            break
    return 0.25*(rating+1)
        
lh['rating'] = lh.apply(to_rating , axis = 1)
lh = lh[['uid','tid','rating','count']]

end = time.time()
print(end - start)

112.72031354904175


In [29]:
lh[:10]

,uid,tid,rating,count
4,0,0,4.75,15
5,0,1,2.50,3
6,0,2,0.25,1
7,0,3,0.25,1
8,0,4,4.50,10
9,0,5,0.25,1
10,0,6,1.50,2
11,0,7,2.50,3
13,0,8,1.50,2
14,0,9,0.25,1


In [30]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))

In [31]:
# minmax

In [32]:
lh = pd.read_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))
lh[:10]

,uid,tid,rating,count
4,0,0,4.75,15
5,0,1,2.50,3
6,0,2,0.25,1
7,0,3,0.25,1
8,0,4,4.50,10
9,0,5,0.25,1
10,0,6,1.50,2
11,0,7,2.50,3
13,0,8,1.50,2
14,0,9,0.25,1


In [33]:
max_list = {}
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    max_value = max(user['count'])
    max_list[i] = max_value

In [34]:
# If the input data is array, try the code below

# for i in train['uid'].unique():
#     user = train[train['uid'] == i]
#     max_value = max(user['count'])
#     print(max_value)
#     normalized_list[i] = [j *5 / max_value for j in counting_list[i]]

# for i in tqdm_notebook(lh['uid'].unique()):
#     user = lh[lh['uid'] == i]
#     max_value = max(user['count'])
#     for index, row in user.iterrows():
#         lh.loc[index, 'count'] = row['count'] *5 / max_value
start = time.time()
def normalize(df):  
    return int(df['count']) *5 / max_list[df['uid']]

lh['minmax'] = lh.apply(normalize , axis = 1)
lh = lh[['uid', 'tid', 'rating', 'minmax', 'count']]

end = time.time()
print(end - start)

15.32011365890503


In [35]:
lh[:10]

,uid,tid,rating,minmax,count
4,0,0,4.75,1.595745,15
5,0,1,2.50,0.319149,3
6,0,2,0.25,0.106383,1
7,0,3,0.25,0.106383,1
8,0,4,4.50,1.063830,10
9,0,5,0.25,0.106383,1
10,0,6,1.50,0.212766,2
11,0,7,2.50,0.319149,3
13,0,8,1.50,0.212766,2
14,0,9,0.25,0.106383,1


In [36]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))